In [16]:
from line_solver import *
import numpy as np
GlobalConstants.set_verbose(VerboseLevel.STD)

In [17]:
# Create network
model = Network('MyNetwork')

# Block 1: nodes
node1 = Delay(model, 'SlowDelay')
node2 = Queue(model, 'FCFSQueue', SchedStrategy.FCFS)
node3 = Queue(model, 'SIROQueue', SchedStrategy.SIRO)  # Service In Random Order
node4 = Queue(model, 'PSQueue', SchedStrategy.PS)      # Processor Sharing
node5 = Queue(model, 'HOLQueue', SchedStrategy.HOL)    # Head-of-Line priority
node6 = Delay(model, 'FastDelay')

In [18]:
# Block 2: classes with priorities
# Priority: 0=low, 1=high - Class2 has higher priority
jobclass1 = ClosedClass(model, 'Class1', 18, node1, 0)  # Low priority
jobclass2 = ClosedClass(model, 'Class2', 18, node1, 1)  # High priority
jobclass3 = ClosedClass(model, 'Class3', 18, node1, 0)  # Low priority

In [19]:
# Set service times - identical at delay stations for fair comparison
# SlowDelay: all classes same service time
node1.set_service(jobclass1, Exp.fit_mean(10.0))
node1.set_service(jobclass2, Exp.fit_mean(10.0))
node1.set_service(jobclass3, Exp.fit_mean(10.0))

# FCFSQueue: different service times per class
node2.set_service(jobclass1, Exp.fit_mean(0.3))
node2.set_service(jobclass2, Exp.fit_mean(0.5))
node2.set_service(jobclass3, Exp.fit_mean(0.6))

# SIROQueue: different service times per class
node3.set_service(jobclass1, Exp.fit_mean(1.1))
node3.set_service(jobclass2, Exp.fit_mean(1.3))
node3.set_service(jobclass3, Exp.fit_mean(1.5))

# PSQueue: different service times per class
node4.set_service(jobclass1, Exp.fit_mean(1.0))
node4.set_service(jobclass2, Exp.fit_mean(1.1))
node4.set_service(jobclass3, Exp.fit_mean(1.9))

# HOLQueue: different service times per class - priority will matter here
node5.set_service(jobclass1, Exp.fit_mean(2.5))
node5.set_service(jobclass2, Exp.fit_mean(1.9))
node5.set_service(jobclass3, Exp.fit_mean(4.3))

# FastDelay: all classes same service time
node6.set_service(jobclass1, Exp.fit_mean(1.0))
node6.set_service(jobclass2, Exp.fit_mean(1.0))
node6.set_service(jobclass3, Exp.fit_mean(1.0))

In [20]:
# Create cyclic routing: SlowDelay -> FCFS -> SIRO -> PS -> HOL -> FastDelay -> SlowDelay
P = model.init_routing_matrix()

# All classes follow the same cyclic route
for jobclass in [jobclass1, jobclass2, jobclass3]:
    P.set(jobclass, jobclass, node1, node2, 1.0)  # SlowDelay -> FCFSQueue
    P.set(jobclass, jobclass, node2, node3, 1.0)  # FCFSQueue -> SIROQueue
    P.set(jobclass, jobclass, node3, node4, 1.0)  # SIROQueue -> PSQueue
    P.set(jobclass, jobclass, node4, node5, 1.0)  # PSQueue -> HOLQueue
    P.set(jobclass, jobclass, node5, node6, 1.0)  # HOLQueue -> FastDelay
    P.set(jobclass, jobclass, node6, node1, 1.0)  # FastDelay -> SlowDelay

model.link(P)

In [21]:
# MVA with seed=23000 to match JAR test
solver = MVA(model, seed=23000, verbose=VerboseLevel.SILENT)
avg_table = solver.avg_table()
print(avg_table)

      Station JobClass        QLen        Util        RespT       ResidT  \
0   SlowDelay   Class1  3.8283e+00  3.8283e+00      10.0000      10.0000   
1   SlowDelay   Class2  5.2631e-04  5.2631e-04      10.0000      10.0000   
2   SlowDelay   Class3  2.2642e+00  2.2642e+00      10.0000      10.0000   
3   FCFSQueue   Class1  1.7244e-01  1.1485e-01       0.4504       0.4504   
4   FCFSQueue   Class2  3.4385e-05  2.6316e-05       0.6533       0.6533   
5   FCFSQueue   Class3  1.6929e-01  1.3585e-01       0.7477       0.7477   
6   SIROQueue   Class1  1.8085e+00  4.2111e-01       4.7240       4.7240   
7   SIROQueue   Class2  2.6497e-04  6.8421e-05       5.0345       5.0345   
8   SIROQueue   Class3  1.1632e+00  3.3963e-01       5.1376       5.1376   
9     PSQueue   Class1  1.8388e+00  3.8283e-01       4.8033       4.8033   
10    PSQueue   Class2  2.7809e-04  5.7895e-05       5.2837       5.2837   
11    PSQueue   Class3  2.0664e+00  4.3019e-01       9.1263       9.1263   
12   HOLQueu

In [22]:
# SSA with seed=23000, samples=10000 to match JAR test
solver = SSA(model, seed=23000, samples=10000, verbose=VerboseLevel.SILENT)
avg_table = solver.avg_table()
print(avg_table)

      Station JobClass        QLen        Util       RespT      ResidT  \
0   SlowDelay   Class1  1.4990e+00  1.4990e+00     10.0000     10.0000   
1   SlowDelay   Class2  3.5141e-02  3.5141e-02     10.0000     10.0000   
2   SlowDelay   Class3  1.4278e+00  1.4278e+00     10.0000     10.0000   
3   FCFSQueue   Class1  6.6749e-02  4.4970e-02      0.4433      0.4433   
4   FCFSQueue   Class2  1.1239e-02  1.7571e-03      3.1932      3.1932   
5   FCFSQueue   Class3  1.1381e-01  8.5667e-02      0.7447      0.7447   
6   SIROQueue   Class1  5.0893e-01  1.6563e-01      3.3011      3.3011   
7   SIROQueue   Class2  1.4018e-01  4.5757e-03     28.3367     28.3367   
8   SIROQueue   Class3  5.2370e-01  2.2923e-01      3.4581      3.4581   
9     PSQueue   Class1  2.8602e-01  1.5417e-01      1.8858      1.8858   
10    PSQueue   Class2  1.7123e-02  5.4415e-03      5.6194      5.6194   
11    PSQueue   Class3  5.1307e-01  2.8774e-01      3.4866      3.4866   
12   HOLQueue   Class1  1.5492e+01  3.